In [62]:
import json
import pandas as pd
from time import sleep
import asyncio
from solana.rpc.async_api import AsyncClient
from datetime import datetime
from itertools import chain
from pydantic import BaseModel, Json
import requests
from typing import Any
from solana.rpc.types import MemcmpOpts

### Marketplace addrs:
#### Thanks Levi for this gist: https://gist.github.com/levicook/34f390073bd57abebc786cda5bec4094

In [2]:
marketplace_mapper = {
    "HZaWndaNWHFDd9Dhk5pqUUtsmoBCqzb1MLu3NAh1VX6B": "AlphaArt",
    "A7p8451ktDCHq5yYaHczeLMYsjRsAkzc3hCXcSrwYHU7": "DigitalEyes",
    "AmK5g2XcyptVLCFESBCJqoSfwV3znGoVYQnqEnaAZKWn": "ExchangeArt",
    "MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8": "MagicEden",
    "CJsLwbP1iu5DuUikHEJnLfANgKy6stB2uFgvBBHoyxwz": "Solanart"
}

marketplace_token_url_mapper = {
    "AlphaArt": "https://alpha.art/t/",
    "DigitalEyes": "https://digitaleyes.market/item/",
    "ExchangeArt": "https://exchange.art/single/",
    "MagicEden": "https://magiceden.io/item-details/",
    "Solanart": "https://solanart.io/search/" # params: ?token=<>
}

marketplace_fetch_id_url = "https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress"

### Setting up http client using genesysgo rpc

In [3]:
SSC_RPC_ENDPOINT = "https://ssc-dao.genesysgo.net"
SOL_EXPLORER_RPC_ENDPOINT = "https://explorer-api.mainnet-beta.solana.com"

In [48]:
sol_client = AsyncClient(SSC_RPC_ENDPOINT)

### Fetching data for all the accounts

In [25]:
SSC_METADATA_JSON_URL = "https://sld-gengo.s3.amazonaws.com"
# "https://sld-gengo.s3.amazonaws.com/3965.json"

In [26]:
SSC_ADDR = "D6wZ5U9onMC578mrKMp5PZtfyc5262426qKsYJW7nT3p"

In [49]:
ssc_account_data = await sol_client.get_account_info(SSC_ADDR)

In [50]:
ssc_account_data

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 113858477},
  'value': {'data': ['', 'base64'],
   'executable': False,
   'lamports': 599009609443,
   'owner': '11111111111111111111111111111111',
   'rentEpoch': 263}},
 'id': 1}

In [51]:
total_txns = await sol_client.get_transaction_count()

In [52]:
total_txns

{'jsonrpc': '2.0', 'result': 48353358883, 'id': 2}

In [53]:
ssc_signatures = await sol_client.get_confirmed_signature_for_address2(SSC_ADDR, limit=500)

In [54]:
len(ssc_signatures["result"])

500

In [55]:
ssc_signatures["result"][-1]

{'blockTime': 1640185393,
 'confirmationStatus': 'finalized',
 'err': None,
 'memo': None,
 'signature': 'eTTkwqKyAwUaAXtWiwr1v9EaQ1NhabjNmDE64Bc7AEdE49eaY9udCkBywFjhTfiFvK16nocbv5pGqx3hb8d4siW',
 'slot': 112758893}

In [22]:
ssc_signatures_prev1 = await sol_client.get_signatures_for_address(
    SSC_ADDR, 
    before="65rTamEzboobANuQAXbfJFPGZtATRZGa5kVCSbPJJNbYzK32HXvoe3gYFSEpiXb23D1hDYrxLWw1RK8dBdVoaeZ7", limit=5)

In [23]:
ssc_signatures_prev1

{'jsonrpc': '2.0', 'result': [], 'id': 6}

In [17]:
def fetch_time_diff(ts_start: datetime, ts_end: datetime, time_format: str = "seconds"):
    return round((ts_end - ts_start).total_seconds(), 2)

async def fetch_txn_batch(rpc_endpoint: str, batch_count: int, cursor_addr: str = None):
    async with AsyncClient(rpc_endpoint) as sol_client:
        ssc_signatures = await sol_client.get_confirmed_signature_for_address2(
            SSC_ADDR,
            limit=batch_count,
            before=cursor_addr
        )
        result = []
        if len(ssc_signatures["result"]) != 0:
            result = ssc_signatures["result"]
        return result

async def fetch_all_txns_for_ssc(rpc_endpoint: str, batch_count: int = 500) -> pd.DataFrame:
    ts_start = datetime.now()
    txn_list = []
    cursor_addr = None
    while True:
        txns = await fetch_txn_batch(rpc_endpoint=rpc_endpoint, 
                                               batch_count=batch_count,
                                               cursor_addr=cursor_addr)
        if len(txns) == 0:
            ts_end = datetime.now()
            break
        cursor_addr = txns[-1]["signature"]
        print(f"Last txn: {cursor_addr} | Len: {len(txns)}")
        txn_list.append(txns)
    final_txn_list = list(chain.from_iterable(txn_list))
    print(f"{len(final_txn_list)} signatures have been fetched in {fetch_time_diff(ts_start=ts_start, ts_end=ts_end)} secs")
    return final_txn_list

In [20]:
results = await fetch_all_txns_for_ssc(rpc_endpoint=SSC_RPC_ENDPOINT)

Last txn: eTTkwqKyAwUaAXtWiwr1v9EaQ1NhabjNmDE64Bc7AEdE49eaY9udCkBywFjhTfiFvK16nocbv5pGqx3hb8d4siW | Len: 500
Last txn: 5VM3meV3RHiHMXp8YEr3AEzE6euKhPhbxADgFGu5uLCE34pYtY62KbDH8GWkkdVXPxwArNYzcJxwPJxcQycARTHN | Len: 500
Last txn: 65T2gWyYaeh9Qjb8yDDKzKhgSQcAG8DPv3CRzXLibgqvQ4wR7bRXXxE38QfDWwtdjjuSLnR1SQgnG1ntj8PQBssp | Len: 500
Last txn: 65rTamEzboobANuQAXbfJFPGZtATRZGa5kVCSbPJJNbYzK32HXvoe3gYFSEpiXb23D1hDYrxLWw1RK8dBdVoaeZ7 | Len: 500
2000 signatures have been fetched in 12.22 secs


### Fetching txn details

In [18]:
results[0]

{'blockTime': 1639886694,
 'confirmationStatus': 'finalized',
 'err': None,
 'memo': None,
 'signature': '44MGvZAXdRYbnwZgGcM6z696NCy5UfvX1XkVgURjafuEx5ujvN4dj31xUAeC5zST46RkKEALcLZU5h2CbVt47eSp',
 'slot': 112226117}

#### 1 SOL = 1_000_000_000 lamports

In [28]:
def convert_lamport_to_sol(amount: float):
    return amount / 1_000_000_000

In [29]:
async def fetch_raw_txn_details(txn: str, rpc_endpoint: str=SSC_RPC_ENDPOINT):
    async with AsyncClient(rpc_endpoint) as sol_client:
        return await sol_client.get_transaction(txn)

In [30]:
deets = await fetch_raw_txn_details(results[850]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

In [31]:
post = deets["result"]["meta"]["postBalances"]

In [32]:
pre = deets["result"]["meta"]["preBalances"]

In [33]:
[convert_lamport_to_sol(bals[0] - bals[1]) for bals in list(zip(pre, post))]

[40.500005,
 0.0,
 -37.66644768,
 0.00144768,
 -0.81,
 0.0,
 -2.025,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [34]:
class SscNftMetadata(BaseModel):
    seller: str
    purchaser: str
    selling_price: float
    seller_cut: float
    marketplace: str
    marketplace_contract_addr: str
    nft_addr: str
    nft_token_id: str # eg. Token id == SSC#4009
    nft_image_url: str
    nft_traits: Any
    nft_marketplace_url: str

In [35]:
async def fetch_token_info(nft_addr: str):
    resp = requests.get(f"{marketplace_fetch_id_url}/{nft_addr}")
    print(resp.url)
    if resp.status_code != 200:
        return None
    nft_metadata = resp.json()["results"]
    return nft_metadata["img"], nft_metadata["attributes"], nft_metadata["title"], nft_metadata["content"] 

In [36]:
async def fetch_txn_details(txn: str, rpc_endpoint: str=SSC_RPC_ENDPOINT):
    formatted_details = {}
    async with AsyncClient(rpc_endpoint) as sol_client:
        txn_details = await sol_client.get_transaction(txn)
        nft_addr = txn_details["result"]["meta"]["postTokenBalances"][0]["mint"]
        pre_balances = txn_details["result"]["meta"]["preBalances"]
        post_balances = txn_details["result"]["meta"]["postBalances"]
        actual_balances = [convert_lamport_to_sol(bals[0] - bals[1]) for bals in list(zip(pre_balances, post_balances))]
        addrs = txn_details["result"]["transaction"]["message"]["accountKeys"]
        balance_list = list(zip(addrs, actual_balances))
        nft_metadata = await fetch_token_info(nft_addr=nft_addr)
        marketplace_name = marketplace_mapper[balance_list[-1][0]]
        metadata = SscNftMetadata(
            seller=balance_list[2][0],
            purchaser=balance_list[0][0],
            selling_price=balance_list[0][1],
            seller_cut=-1 * (balance_list[2][1]),
            marketplace=marketplace_name,
            marketplace_contract_addr=balance_list[-1][0],
            nft_addr=nft_addr,
            nft_token_id=nft_metadata[2],
            nft_image_url=nft_metadata[0],
            nft_traits=nft_metadata[1],
            nft_marketplace_url=f"{marketplace_token_url_mapper[marketplace_name]}{nft_addr}"
        )
        return metadata.dict()

In [38]:
await fetch_txn_details(results[850]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/6yzxN42EzAp9uJUxwafi9TAWNqY4rMKxhpG3AZYTDyXc


{'seller': 'CznocEx6VgqqsBZhBJZwweQjxPZEwxZTLhSysNbhqJwm',
 'purchaser': '8fcVsdDKtPRwNjwgMHoxxFt2VW4mw2d5PduDyjgANc1j',
 'selling_price': 40.500005,
 'seller_cut': 37.66644768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '6yzxN42EzAp9uJUxwafi9TAWNqY4rMKxhpG3AZYTDyXc',
 'nft_token_id': 'Shadowy Super Coder #8250',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/8250.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Magenta'},
  {'trait_type': 'Base Model', 'value': 'Tan'},
  {'trait_type': 'Desk', 'value': 'Silver Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Goo'},
  {'trait_type': 'Monitor 2', 'value': 'Linux'},
  {'trait_type': 'Monitor 3', 'value': 'Hacked'},
  {'trait_type': 'Mouth', 'value': 'Purple Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Tron Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Java'}],
 'nft_marketplace_url': 'https://magiceden.io/item-details/6

In [39]:
await fetch_txn_details(results[100]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/54NY9idmKtBvLcZjfmxQkyAJRMwbTN4YD7pDZ6ByVCgM


{'seller': '6GQErHmLEx37atnLuCSqhTRntz5vkkj3gBzQGzYsuenZ',
 'purchaser': '3vnDhqKYCWMKjAe9Y6K5d4NCz9zuzTarGX1mjew5Ca9c',
 'selling_price': 72.000005,
 'seller_cut': 66.96144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '54NY9idmKtBvLcZjfmxQkyAJRMwbTN4YD7pDZ6ByVCgM',
 'nft_token_id': 'Shadowy Super Coder #3705',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/3705.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Gold'},
  {'trait_type': 'Base Model', 'value': 'Tan'},
  {'trait_type': 'Desk', 'value': 'Bronze Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Trading Dark'},
  {'trait_type': 'Monitor 2', 'value': 'Matrix'},
  {'trait_type': 'Monitor 3', 'value': 'Trading Dark'},
  {'trait_type': 'Mouth', 'value': 'Red Smile'},
  {'trait_type': 'Hoodie', 'value': 'Cat-Ear Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Rust'}],
 'nft_marketplace_url': 'https://magiceden.io/i

In [42]:
await fetch_txn_details(results[1998]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

TypeError: 'NoneType' object is not subscriptable

In [43]:
await fetch_txn_details(results[200]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/CAWzZt8AVb8xD6EK1JhkRHpq58U4e31oDGWvfjzLzCnz


{'seller': '3zKPWGbYNaPoBv2rqoAJ9QTQxUcyMMadPFCmtj3273wU',
 'purchaser': 'D9WydY3gGUoJjPnRmrbV5jfAnviJ5TCReMHeaBogZ9dg',
 'selling_price': 84.880005,
 'seller_cut': 78.93984768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': 'CAWzZt8AVb8xD6EK1JhkRHpq58U4e31oDGWvfjzLzCnz',
 'nft_token_id': 'Shadowy Super Coder #3708',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/3708.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Green'},
  {'trait_type': 'Base Model', 'value': 'Pale'},
  {'trait_type': 'Desk', 'value': 'Bronze Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Glitch Two'},
  {'trait_type': 'Monitor 2', 'value': 'Access Granted'},
  {'trait_type': 'Monitor 3', 'value': 'Glitch One'},
  {'trait_type': 'Mouth', 'value': 'Purple Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Squid Game Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Fortran'}],
 'nft_marketplace_url': 'https:/

In [44]:
await fetch_txn_details(results[10]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/5vijUSg4ZUTAtD5S2n4r1jBEsG3KT1ZiE1NVj2k6jB1G


{'seller': '6GQErHmLEx37atnLuCSqhTRntz5vkkj3gBzQGzYsuenZ',
 'purchaser': '3gc7tTph3waXqfhox8Ln9m4BkrzSvSQQYsxdd9U7Btk9',
 'selling_price': 80.000005,
 'seller_cut': 74.40144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '5vijUSg4ZUTAtD5S2n4r1jBEsG3KT1ZiE1NVj2k6jB1G',
 'nft_token_id': 'Shadowy Super Coder #5464',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/5464.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Light-Yellow'},
  {'trait_type': 'Base Model', 'value': 'Pale'},
  {'trait_type': 'Desk', 'value': 'Silver Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Matrix'},
  {'trait_type': 'Monitor 2', 'value': 'Linux'},
  {'trait_type': 'Monitor 3', 'value': 'Matrix'},
  {'trait_type': 'Mouth', 'value': 'Pink Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Shark Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Java'}],
 'nft_marketplace_url': 'https://magiceden.io/item-d

In [45]:
await fetch_txn_details(results[0]["signature"], rpc_endpoint=SSC_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/4z6seZ1EQbUJ5zyEcF85TrhECp3QeFkzh9Zt745YMv74


{'seller': '6GQErHmLEx37atnLuCSqhTRntz5vkkj3gBzQGzYsuenZ',
 'purchaser': '9uvwcE7xKqbhTcuQMyU6oJLEQnqD23VaJ4xvEaKpB7ya',
 'selling_price': 80.000005,
 'seller_cut': 74.40144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '4z6seZ1EQbUJ5zyEcF85TrhECp3QeFkzh9Zt745YMv74',
 'nft_token_id': 'Shadowy Super Coder #4817',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/4817.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Dark-Red'},
  {'trait_type': 'Base Model', 'value': 'Chestnut'},
  {'trait_type': 'Desk', 'value': 'Bronze Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Access Granted'},
  {'trait_type': 'Monitor 2', 'value': 'Barret'},
  {'trait_type': 'Monitor 3', 'value': 'Future Finance'},
  {'trait_type': 'Mouth', 'value': 'Pink Smile'},
  {'trait_type': 'Hoodie', 'value': 'Neon Solana Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Go'}],
 'nft_marketplace_url': 'https:

In [46]:
await fetch_txn_details("5yZNYqNi13NDTzW5BdDHucJpeG6SumL1y4G3AwfBiTgVdJpb1d12RmaisSM7yi3r1nHuaLLmQQHvLwkSrzr3Z5TG")

TypeError: 'NoneType' object is not subscriptable

### Get all mint addresses

Above method to get all txns is probably very great to fetch txns, but inorder to get all the mints we need to use the `getProgramAccounts` function

In [56]:
await sol_client.get_program_accounts(SSC_ADDR)

{'jsonrpc': '2.0', 'result': [], 'id': 4}

In [72]:
# url="https://api.mainnet-beta.solana.com"
# url="https://solana-api.projectserum.com"
METADATA_PROGRAM_ID="metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s"
MAX_NAME_LENGTH = 32;
MAX_URI_LENGTH = 200;
MAX_SYMBOL_LENGTH = 10;
MAX_CREATOR_LEN = 32 + 1 + 1;

creatorIndex=0
creatorAddress="Bhr9iWx7vAZ4JDD5DVSdHxQLqG9RvCLCSXvu6yC4TF6c"

payload=json.dumps({
    "jsonrpc":"2.0",
    "id":1,
    "method":"getProgramAccounts",
    "params":[
        METADATA_PROGRAM_ID,
        {
            "encoding":"jsonParsed",
            "filters":[
                {
                    "memcmp": {
                        "offset":
                            1 + # key
                            32 + # update auth
                            32 + # mint
                            4 + # name string length
                            MAX_NAME_LENGTH + # name
                            4 + # uri string length
                            MAX_URI_LENGTH + # uri
                            4 + # symbol string length
                            MAX_SYMBOL_LENGTH + # symbol
                            2 + # seller fee basis points
                            1 + # whether or not there is a creators vec
                            4 + # creators vec length
                            creatorIndex * MAX_CREATOR_LEN,
                        "bytes": creatorAddress
                    },
                },
            ],            
        }
    ]
})

headers={"content-type":"application/json","cache-control":"no-cache"}
response=requests.request("POST", SSC_RPC_ENDPOINT,data=payload,headers=headers)
data=response.json()["result"]
print("total records:",len(data))

total records: 0


In [73]:
data

[]

In [64]:
# pubkey: update mint authority on solscan (go to any token and check the key)
# 

memcmp_opts = [MemcmpOpts(offset=4, bytes="Bhr9iWx7vAZ4JDD5DVSdHxQLqG9RvCLCSXvu6yC4TF6c")]
data = await sol_client.get_program_accounts(pubkey="Bhr9iWx7vAZ4JDD5DVSdHxQLqG9RvCLCSXvu6yC4TF6c",
                               encoding="jsonParsed",
                               memcmp_opts=memcmp_opts)

In [67]:
from solana.rpc.types import TokenAccountOpts

In [68]:
await sol_client.get_token_accounts_by_owner("Bhr9iWx7vAZ4JDD5DVSdHxQLqG9RvCLCSXvu6yC4TF6c", opts=[TokenAccountOpts(mintprogram_id="metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")])

AttributeError: 'list' object has no attribute 'mint'

In [65]:
data

{'jsonrpc': '2.0', 'result': [], 'id': 5}